In [ ]:
import os
import stitching
import cv2
import matplotlib.pyplot as plt
import numpy as np
import stitching
import time

In [ ]:
ROOT_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(ROOT_PATH, "data")

In [ ]:
#left_test_image_path = os.path.join(DATA_PATH, "keparoi_left_frame_aligned.jpg")
#right_test_image_path = os.path.join(DATA_PATH, "keparoi_right_frame_aligned.jpg")

left_test_image_path = "/media/jakki/Seagate_Expansion_Drive/keparoi/2022/matsi_30082022/meow_tmp_matsi_30082022"
right_test_image_path = "/media/jakki/Seagate_Expansion_Drive/keparoi/2022/matsi_30082022/meow_tmp_matsi_30082022"

In [ ]:
real_K = np.load(os.path.join(DATA_PATH, "camera_matrix.npy"))
print(real_K)

In [ ]:
stitcher = stitching.Stitcher(blend_strength=20, try_use_gpu=True, warper_type="mercator")
panorama = stitcher.stitch([left_test_image_path, right_test_image_path])
fix_colors = cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(32,32))
plt.imshow(fix_colors)
plt.show()

In [ ]:
cv2.imwrite(os.path.join(DATA_PATH, "stitching_library_mercator.jpg"), panorama)

## Test tutorial

In [ ]:
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np

def plot_image(img, figsize_in_inches=(5,5)):
    fig, ax = plt.subplots(figsize=figsize_in_inches)
    ax.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    plt.show()
    
def plot_images(imgs, figsize_in_inches=(5,5)):
    fig, axs = plt.subplots(1, len(imgs), figsize=figsize_in_inches)
    for col, img in enumerate(imgs):
        axs[col].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    plt.show()

In [ ]:
left_test_image_path = os.path.join(DATA_PATH, "keparoi_left_frame_aligned.jpg")
right_test_image_path = os.path.join(DATA_PATH, "keparoi_right_frame_aligned.jpg")
left_undistorted_test_image_path = os.path.join(DATA_PATH, "left_image_undistorted.jpg")
right_undistorted_test_image_path = os.path.join(DATA_PATH, "right_image_undistorted.jpg")

In [ ]:
from stitching.images import Images

#image_names = [left_test_image_path, right_test_image_path]
image_names = [left_undistorted_test_image_path, right_undistorted_test_image_path]

images = Images.of(image_names)

medium_imgs = list(images.resize(Images.Resolution.MEDIUM))
low_imgs = list(images.resize(Images.Resolution.LOW))
final_imgs = list(images.resize(Images.Resolution.FINAL))

In [ ]:
plot_images(low_imgs, (20,20))

In [ ]:
original_size = images.sizes[0]
medium_size = images.get_image_size(medium_imgs[0])
low_size = images.get_image_size(low_imgs[0])
final_size = images.get_image_size(final_imgs[0])

print(f"Original Size: {original_size}  -> {'{:,}'.format(np.prod(original_size))} px ~ 1 MP")
print(f"Medium Size:   {medium_size}  -> {'{:,}'.format(np.prod(medium_size))} px ~ 0.6 MP")
print(f"Low Size:      {low_size}   -> {'{:,}'.format(np.prod(low_size))} px ~ 0.1 MP")
print(f"Final Size:    {final_size}  -> {'{:,}'.format(np.prod(final_size))} px ~ 1 MP")

In [ ]:
from stitching.feature_detector import FeatureDetector

finder = FeatureDetector()
features = [finder.detect_features(img) for img in medium_imgs]
keypoints_center_img = finder.draw_keypoints(medium_imgs[1], features[1])

In [ ]:
plot_image(keypoints_center_img, (15,10))

In [ ]:
from stitching.feature_matcher import FeatureMatcher

matcher = FeatureMatcher()
matches = matcher.match_features(features)

In [ ]:
matcher.get_confidence_matrix(matches)

In [ ]:
all_relevant_matches = matcher.draw_matches_matrix(medium_imgs, features, matches, conf_thresh=1, 
                                                   inliers=True, matchColor=(0, 255, 0))

for idx1, idx2, img in all_relevant_matches:
    print(f"Matches Image {idx1+1} to Image {idx2+1}")
    plot_image(img, (20,10))

In [ ]:
from stitching.subsetter import Subsetter

subsetter = Subsetter()
dot_notation = subsetter.get_matches_graph(images.names, matches)
print(dot_notation)

In [ ]:
indices = subsetter.get_indices_to_keep(features, matches)

medium_imgs = subsetter.subset_list(medium_imgs, indices)
low_imgs = subsetter.subset_list(low_imgs, indices)
final_imgs = subsetter.subset_list(final_imgs, indices)
features = subsetter.subset_list(features, indices)
matches = subsetter.subset_matches(matches, indices)

images.subset(indices)

print(images.names)
print(matcher.get_confidence_matrix(matches))

In [ ]:
from stitching.camera_estimator import CameraEstimator
from stitching.camera_adjuster import CameraAdjuster
from stitching.camera_wave_corrector import WaveCorrector

camera_estimator = CameraEstimator()
camera_adjuster = CameraAdjuster()
wave_corrector = WaveCorrector()

cameras = camera_estimator.estimate(features, matches)
cameras = camera_adjuster.adjust(features, matches, cameras)
cameras = wave_corrector.correct(cameras)

In [ ]:
from stitching.warper import Warper

warper = Warper(warper_type="transverseMercator")

In [ ]:
warper.set_scale(cameras)

In [ ]:
low_sizes = images.get_scaled_img_sizes(Images.Resolution.LOW)
camera_aspect = images.get_ratio(Images.Resolution.MEDIUM, Images.Resolution.LOW)  # since cameras were obtained on medium imgs

warped_low_imgs = list(warper.warp_images(low_imgs, cameras, camera_aspect))
warped_low_masks = list(warper.create_and_warp_masks(low_sizes, cameras, camera_aspect))
low_corners, low_sizes = warper.warp_rois(low_sizes, cameras, camera_aspect)

In [ ]:
final_sizes = images.get_scaled_img_sizes(Images.Resolution.FINAL)
camera_aspect = images.get_ratio(Images.Resolution.MEDIUM, Images.Resolution.FINAL)

warped_final_imgs = list(warper.warp_images(final_imgs, cameras, camera_aspect))
warped_final_masks = list(warper.create_and_warp_masks(final_sizes, cameras, camera_aspect))
final_corners, final_sizes = warper.warp_rois(final_sizes, cameras, camera_aspect)

In [ ]:
plot_images(warped_low_imgs, (10,10))
plot_images(warped_low_masks, (10,10))

In [ ]:
print(final_corners)
print(final_sizes)

In [ ]:
from stitching.timelapser import Timelapser

timelapser = Timelapser('as_is')
timelapser.initialize(final_corners, final_sizes)

for img, corner in zip(warped_final_imgs, final_corners):
    timelapser.process_frame(img, corner)
    frame = timelapser.get_frame()
    plot_image(frame, (10,10))

In [ ]:
from stitching.cropper import Cropper

cropper = Cropper()

In [ ]:
mask = cropper.estimate_panorama_mask(warped_low_imgs, warped_low_masks, low_corners, low_sizes)
plot_image(mask, (5,5))

In [ ]:
lir = cropper.estimate_largest_interior_rectangle(mask)
print(lir)

In [ ]:
plot = lir.draw_on(mask, size=2)
plot_image(plot, (5,5))

In [ ]:
low_corners = cropper.get_zero_center_corners(low_corners)
rectangles = cropper.get_rectangles(low_corners, low_sizes)

plot = rectangles[1].draw_on(plot, (0, 255, 0), 2)  # The rectangle of the center img
plot_image(plot, (5,5))

In [ ]:
overlap = cropper.get_overlap(rectangles[1], lir)
plot = overlap.draw_on(plot, (255, 0, 0), 2)
plot_image(plot, (5,5))

In [ ]:
intersection = cropper.get_intersection(rectangles[1], overlap)
plot = intersection.draw_on(warped_low_masks[1], (255, 0, 0), 2)
plot_image(plot, (2.5,2.5))

In [ ]:
cropper.prepare(warped_low_imgs, warped_low_masks, low_corners, low_sizes)

cropped_low_masks = list(cropper.crop_images(warped_low_masks))
cropped_low_imgs = list(cropper.crop_images(warped_low_imgs))
low_corners, low_sizes = cropper.crop_rois(low_corners, low_sizes)

lir_aspect = images.get_ratio(Images.Resolution.LOW, Images.Resolution.FINAL)  # since lir was obtained on low imgs
cropped_final_masks = list(cropper.crop_images(warped_final_masks, lir_aspect))
cropped_final_imgs = list(cropper.crop_images(warped_final_imgs, lir_aspect))
final_corners, final_sizes = cropper.crop_rois(final_corners, final_sizes, lir_aspect)

In [ ]:
timelapser = Timelapser('as_is')
timelapser.initialize(final_corners, final_sizes)

for img, corner in zip(cropped_final_imgs, final_corners):
    timelapser.process_frame(img, corner)
    frame = timelapser.get_frame()
    plot_image(frame, (10,10))

In [ ]:
from stitching.seam_finder import SeamFinder

seam_finder = SeamFinder()

seam_masks = seam_finder.find(cropped_low_imgs, low_corners, cropped_low_masks)
seam_masks = [seam_finder.resize(seam_mask, mask) for seam_mask, mask in zip(seam_masks, cropped_final_masks)]

seam_masks_plots = [SeamFinder.draw_seam_mask(img, seam_mask) for img, seam_mask in zip(cropped_final_imgs, seam_masks)]
plot_images(seam_masks_plots, (15,10))

In [ ]:
from stitching.exposure_error_compensator import ExposureErrorCompensator

compensator = ExposureErrorCompensator()

compensator.feed(low_corners, cropped_low_imgs, cropped_low_masks)

compensated_imgs = [compensator.apply(idx, corner, img, mask) 
                    for idx, (img, mask, corner) 
                    in enumerate(zip(cropped_final_imgs, cropped_final_masks, final_corners))]

In [ ]:
from stitching.blender import Blender

blender = Blender()
blender.prepare(final_corners, final_sizes)
for img, mask, corner in zip(compensated_imgs, seam_masks, final_corners):
    blender.feed(img, mask, corner)
panorama, _ = blender.blend()

In [ ]:
plot_image(panorama, (20,20))

In [ ]:
blended_seam_masks = seam_finder.blend_seam_masks(seam_masks, final_corners, final_sizes)
plot_image(blended_seam_masks, (5,5))

In [ ]:
plot_image(seam_finder.draw_seam_lines(panorama, blended_seam_masks, linesize=3), (15,10))
plot_image(seam_finder.draw_seam_polygons(panorama, blended_seam_masks), (15,10))

In [ ]:
panorama.shape

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(panorama)
plt.show()

In [ ]:
h, w, d = panorama.shape
print(h)
print(w)
print(d)

final_height, final_width = 1626, 2890
x, y = 1800, 0
crop_img = panorama[y:y+final_height,x:final_width+x]
plt.figure(figsize=(16,16))
plt.imshow(crop_img)
plt.show()

In [ ]:
camera_matrix = np.load(os.path.join(DATA_PATH, 'camera_matrix.npy'))
distortions = np.load(os.path.join(DATA_PATH, 'camera_distortion.npy'))
undistorted_camera_matrix = np.load(os.path.join(DATA_PATH, 'undistorted_camera_matrix.npy'))

h, w = crop_img.shape[0:2]
undistorted_image = cv2.undistort(crop_img, camera_matrix, distortions, None, undistorted_camera_matrix)

plt.imshow(undistorted_image)
plt.show()